In [3]:
from io import StringIO

import matplotlib.pyplot as plt
from scipy.io import arff
import seaborn as sns
from loguru import logger
import yaml

from datetime import datetime
import polars as pl
import pandas as pd
import numpy as np
import sys
import os
from pathlib import Path
sys.path.append(str(Path.cwd().parent))

# MODEL
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import plot_model
from sklearn.preprocessing import (
    LabelEncoder, 
    StandardScaler,
    label_binarize
)
from sklearn.metrics import (
    classification_report, 
    confusion_matrix, 
    accuracy_score,
    precision_recall_fscore_support,
    balanced_accuracy_score,
    roc_auc_score,
    roc_curve
)

# Save variables for model
import joblib
import json


import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2

# PERSONAL FUNCTIONS
from utils import *
from models.main import *
from models.optimizer import ViterbiLiteDecoder
from functions.windows import create_feature_windows # creación de ventanas e ingenieria de características
from functions.build_window_raw import create_raw_windows_250_timesteps_robust

In [4]:
dict_preds = {
    'Eat': {'y': [], 'y_pred': []},
    'Stand': {'y': [], 'y_pred': []},
    'Walk': {'y': [], 'y_pred': []},
    'Sit': {'y': [], 'y_pred': []},
    'Type': {'y': [], 'y_pred': []}
}

In [5]:
dic_result = {
            'Eat': {'Accuracy' : [], 'F1-score': [], 'Precision': [], 'Recall': []}, 
            'Walk': {'Accuracy' : [], 'F1-score': [], 'Precision': [], 'Recall': []},
            'Sit': {'Accuracy' : [], 'F1-score': [], 'Precision': [], 'Recall': []},
            'Stand': {'Accuracy' : [], 'F1-score': [], 'Precision': [], 'Recall': []},
            'Type': {'Accuracy' : [], 'F1-score': [], 'Precision': [], 'Recall': []}
        }

In [7]:
model = keras.models.load_model(r"F:\UPC\Tesis\HARbit-Model\src\models\version\sensors\accel_cnn-lstm_wisdm_91_cluster_user.h5")
label_encoder = joblib.load(r'F:\UPC\Tesis\HARbit-Model\src\models\meta\sensors\accel_cnn-lstm_wisdm_91_cluster_user.joblib')

In [11]:
path_base = r"F:\UPC\Tesis\HARbit-Model\src\data\real-data"
archive_date = [archive for archive in os.listdir(r"F:\UPC\Tesis\HARbit-Model\src\data\real-data") if '.json' not in archive]


for data in archive_date:
    path_data = os.path.join(path_base, data)
    file_data = os.listdir(path_data)

    target = data.split('-')[0].title()
    for file in file_data:

        file_path = os.path.join(path_data, file)
        
        with open(file_path, 'rb') as file:
            data = json.load(file)

        accel_df = data['accel']

        accel_temp = pl.DataFrame(accel_df)
        
        accel_temp = accel_temp.with_columns(pl.lit('A').alias('Usuario'))

        accel_temp = accel_temp.with_columns(pl.lit(target).alias('gt'))

        df_accel = normalize_columns(accel_temp,
                            user_col_name  = "Usuario", 
                            timestamp_col_name = "timestamp", 
                            label_col_name = "gt", 
                            x_col_name = "x", 
                            y_col_name = "y", 
                            z_col_name = "z"
                        )
        
        df_accel = convert_timestamp(df_accel)

        features_accel = create_feature_windows(df_accel, window_seconds=5, overlap_percent=50)

        X_all, y_all, _, le = prepare_features_for_cnn_lstm_sequences(
            features_accel,
            group_size = 8, 
            step_size = 1
        )

        y_all = le.inverse_transform(y_all)
        y_all = label_encoder.transform(y_all)

        num_classes = len(label_encoder.classes_)

        # Predicciones
        y_pred = model.predict(X_all)
        y_pred_classes = np.argmax(y_pred, axis=1)

        # Comparar con y_real (asegúrate de que y_real tenga etiquetas 0..N-1)
        acc = accuracy_score(y_all, y_pred_classes)
        
        precision, recall, f1, _ = precision_recall_fscore_support(
                y_all, y_pred_classes, average='macro', zero_division=0
            )
        
        dic_result[target]['Accuracy'].append(acc*100)
        dic_result[target]['F1-score'].append(f1*100)
        dic_result[target]['Precision'].append(precision*100)
        dic_result[target]['Recall'].append(recall*100)

        dict_preds[target]['y'].extend(y_all)
        dict_preds[target]['y_pred'].extend(y_pred_classes)        

✅ Secuencias creadas: (7, 8, 68)
  Num features: 68
  Clases: ['Eat']
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 644ms/step
✅ Secuencias creadas: (24, 8, 68)
  Num features: 68
  Clases: ['Sit']
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 506ms/step
✅ Secuencias creadas: (17, 8, 68)
  Num features: 68
  Clases: ['Sit']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
✅ Secuencias creadas: (17, 8, 68)
  Num features: 68
  Clases: ['Sit']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
✅ Secuencias creadas: (22, 8, 68)
  Num features: 68
  Clases: ['Sit']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
✅ Secuencias creadas: (16, 8, 68)
  Num features: 68
  Clases: ['Sit']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
✅ Secuencias creadas: (16, 8, 68)
  Num features: 68
  Clases: ['Sit']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
✅ Secuencias creadas: (24, 8, 68)
  Num features: 68
  Clases: ['Sit']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
✅ Secuencias creadas: (13, 8, 68)
  Num features: 68
  Clases: ['Sit']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
✅ Secuencias cread

In [14]:
from sklearn.metrics import classification_report

y_true_global = []
y_pred_global = []

for activity in dict_preds.keys():
    y_true_global.extend(dict_preds[activity]['y'])
    y_pred_global.extend(dict_preds[activity]['y_pred'])

labels = np.unique(y_true_global)  # clases presentes en los datos
target_names = label_encoder.inverse_transform(labels)

print(classification_report(
    y_true_global, y_pred_global,
    labels=labels,
    target_names=target_names
))

              precision    recall  f1-score   support

         Eat       0.01      0.29      0.02         7
         Sit       0.27      0.08      0.12       164
       Stand       0.19      0.15      0.17       149
        Type       0.00      0.00      0.00       227
        Walk       0.80      0.58      0.68       932

   micro avg       0.54      0.39      0.45      1479
   macro avg       0.25      0.22      0.20      1479
weighted avg       0.56      0.39      0.46      1479

